In [1]:
import numpy as np
import cv2
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import load_img
from matplotlib import pyplot as plt



In [ ]:
img = load_img('data/train/annotations/task-1-annotation-1-by-1-tag-House-0.png',target_size=(332,414)) # target_size=(166,207))


plt.imshow(img)

In [2]:
class InputStream:
    def __init__(self, data):
        self.data = data
        self.i = 0

    def read(self, size):
        out = self.data[self.i:self.i + size]
        self.i += size
        return int(out, 2)

In [3]:
def access_bit(data, num):
    """ from bytes array to bits by num position
    """
    base = int(num // 8)
    shift = 7 - int(num % 8)
    return (data[base] & (1 << shift)) >> shift

def bytes2bit(data):
    """ get bit string from bytes data
    """
    return ''.join([str(access_bit(data, i)) for i in range(len(data) * 8)])

In [4]:
def decode_rle(rle, original_height, original_width, print_params: bool = False):
    """ from LS RLE to numpy uint8 3d image [width, height, channel]
    
    Args:
        print_params (bool, optional): If true, a RLE parameters print statement is suppressed
    """
    input = InputStream(bytes2bit(rle))
    num = input.read(32)
    word_size = input.read(5) + 1
    rle_sizes = [input.read(4) + 1 for _ in range(4)]
    
    if print_params:
        print('RLE params:', num, 'values', word_size, 'word_size', rle_sizes, 'rle_sizes')
        
    i = 0
    out = np.zeros(num, dtype=np.uint8)
    while i < num:
        x = input.read(1)
        j = i + 1 + input.read(rle_sizes[input.read(2)])
        if x:
            val = input.read(word_size)
            out[i:j] = val
            i = j
        else:
            while i < j:
                val = input.read(word_size)
                out[i] = val
                i += 1
    out = np.reshape(out, [original_height, original_width, 4])[:,:,3]
    out[out>0] = 1
    out = np.reshape(out,[original_height, original_width, 1])
    return out

In [5]:
import os
import json

In [6]:
file = open('data/train/annotations.json')

data = json.load(file)

In [7]:
info = []
for item in data:
    file_name = item['image']
    mask = decode_rle(item['tag'][0]['rle'], item['tag'][0]['original_height'], item['tag'][0]['original_width'])
    info.append((file_name, mask))

In [16]:
batch_size = 32
image_size = (256, 256)
batch_annotations = info[0 : batch_size]
x = np.zeros((batch_size,) + image_size + (3,), dtype="float32")
y = np.zeros((batch_size,) + image_size + (1,), dtype="uint8")
print(y.shape)
for j, data in enumerate(batch_annotations):
    img = cv2.resize(cv2.cvtColor(cv2.imread(f'data/images/{data[0]}'), cv2.COLOR_BGR2RGB), image_size)
    print(img.shape)
    x[j] = img
    y[j] = np.expand_dims(cv2.resize(data[1], image_size),2)

(32, 256, 256, 1)
(256, 256, 3)
(528, 297, 1)
(256, 256, 1)


ValueError: could not broadcast input array from shape (256,256) into shape (256,256,1)

In [ ]:
g = 0
for a in range(batch_size):
    c = np.random.rand()
    if(c > 0.5):
        g+=1

print(batch_size/g)

In [ ]:
print(x.shape)
print(y.shape)
plt.subplot(121), plt.imshow(x[0])
plt.subplot(122), plt.imshow(y[0], cmap='gray')

In [ ]:
def blur(image):
    sigmaX = random.random()
    image_ = cv2.GaussianBlur(image,(5,5),sigmaX)
    return image_

def flip(image, mask):
    image_ = np.fliplr(image)
    mask_ = np.fliplr(mask)
    return image_, mask_

def rotate(image, mask):
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    
    angle = random.randint(1, 25)
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    image_ = cv2.warpAffine(image, M, (w, h))
    mask_ =  cv2.warpAffine(mask, M, (w, h))
    return image_, mask_

In [ ]:
img_num = 0
img_name = info[img_num][0]
# img = load_img(f'data/train/images/{img_name}')
img = cv2.cvtColor(cv2.imread(f'data/train/images/{img_name}'), cv2.COLOR_BGR2RGB)
mask = info[img_num][1]

# img, mask = flip(img,mask)
# img = blur(img)
# img, mask = rotate(img,mask)

print(img_name)
plt.subplot(121), plt.imshow(img)
plt.subplot(122), plt.imshow(mask, cmap='gray')